# Webscraper (for reddit.com, project 3)
(Example: [https://www.youtube.com/watch?v=AcrjEWsMi_E](https://www.youtube.com/watch?v=AcrjEWsMi_E))

In [1]:
# Last Updated: 2021-04-24, 00:14

import datetime
import os

import requests
import pandas as pd
import numpy as np

In [2]:
# GLOBALS:
#     - base_url

# Create function:
#     ARGS:   is_submission (or content, BOOL)
#             params, e.g. { subreddit, size, before }
# 
# DO, 10 or 20 times (once for the first few times):
#                 Get100, AMA, alternate and get 100 Ask Reddit
#                 Export each to (dated/timed) csv (or same, but add 'y' column, e.g. AMA or AskReddit)
#                 get 'created_utc' value for each, set 'before' with it for next iter
#                 
#                 
#                 Make something that records last val, so not pulling redundant data
#                 probably manual at first. later maybe use a text file for each.


In [3]:
# GLOBALS
BASE_URL = 'https://api.pushshift.io/reddit/search/'

In [120]:
def call_scraper(size): #, one_only = None):
    submission_comment = ['submission', 'comment']
    sub_reddit = ['AMA', 'AskReddit']
    b4 = []
    
    # read in before.csv values as a list, and apply them 
    old_before_list = read_in_before_csv()
    c = 0    # counter
    
#     Need to figure out how to deal with before vals in this situation
#     if(one_only.lower() == 'submission'):
#         submission_comment = 'submission'
#     elif(one_only.lower() == 'comment'):
#         submission_comment = 'comment'
    
    for sr in sub_reddit:
        for sc in submission_comment:
            before = 0
            if(c < len(old_before_list)):
                before = old_before_list[c]
            url = BASE_URL + sc
            b4v = start_scraping(url, sr, sc, size, before)
            b4.append(b4v)
            c += 1
    
    df_before = make_before_vals_df(b4[0], b4[1], b4[2], b4[3])
    df_before.to_csv('./before.csv', index=False)
    
    return

In [121]:
def read_in_before_csv():
    b4v = pd.read_csv('before.csv')
    b4v_s = b4v['before_value'].str.replace('_', '').apply(int)
    print('READ IN BEFORE VALS: ')
    print(b4v_s)
    return b4v_s

In [122]:
def start_scraping(url, sub_reddit, sub_com, size=100, before=0):
    status, req = set_params_and_request(url, sub_reddit, size, before)
    
    print(f'{sub_reddit} STATUS ({sub_com}): {status}')
    data = req.json()['data']
    df_req = pd.DataFrame(data)
    write_to_csv(df_req, sub_reddit, sub_com)
    
    before = data[-1]['created_utc']
    return before

In [123]:
def first_start_scraping(size=100, before=0):
    url_s = BASE_URL + 'submission'
    url_c = BASE_URL + 'comment'
    sub_reddit = ['AMA', 'AskReddit']
    
    status_ama_s, req_ama_s = set_params_and_request(url_s, sub_reddit[0], size, before)
    status_ama_c, req_ama_c = set_params_and_request(url_c, sub_reddit[0], size, before)
    status_ar_s, req_ar_s = set_params_and_request(url_s, sub_reddit[1], size, before)
    status_ar_c, req_ar_c = set_params_and_request(url_c, sub_reddit[1], size, before)
    
    print(f'AMA STATUS (Submission): {status_ama_s}')
    print(f'AMA STATUS (Comments):   {status_ama_c}')
    print(f' AR STATUS (Submission): {status_ar_s}')
    print(f' AR STATUS (Comments):   {status_ar_c}')

    # GET JSON: data = req.json()
    # CONVERT JSON TO  DATA: output = data['data']
    data_req_ama_s = req_ama_s.json()['data']
    data_req_ama_c = req_ama_c.json()['data']
    data_req_ar_s = req_ar_s.json()['data']
    data_req_ar_c = req_ar_c.json()['data']
    
    # get 'created_utc' for 'before' field
    before_req_ama_s = data_req_ama_s[-1]['created_utc']
    before_req_ama_c = data_req_ama_c[-1]['created_utc']
    before_req_ar_s = data_req_ar_s[-1]['created_utc']
    before_req_ar_c = data_req_ar_c[-1]['created_utc']
    
    # Set to dataframe: 
    df_req_ama_s = pd.DataFrame(data_req_ama_s)
    df_req_ama_c = pd.DataFrame(data_req_ama_c)
    df_req_ar_s = pd.DataFrame(data_req_ar_s)
    df_req_ar_c = pd.DataFrame(data_req_ar_c)
    
    # output to csv file somewhere
    write_to_csv(df_req_ama_s, 'ama', 'submission')
    write_to_csv(df_req_ama_c, 'ama', 'comments')
    write_to_csv(df_req_ar_s, 'ar', 'submission')
    write_to_csv(df_req_ar_c, 'ar', 'comments')
    
    #output Before Values:
    df_before = make_before_vals_df(before_req_ama_s, before_req_ama_c, before_req_ar_s, before_req_ar_c)
    write_to_csv(df_before, 'before', 'values')
    
    return

In [124]:
def set_params_and_request(url, sr, size, before):
    params = {}

    if(before <=0):
        params = {
            'subreddit': sr,
            'size': size
        }
    else:
        params = {
            'subreddit': sr,
            'size': size,
            'before': before    # 'created_utc'
        }
    return get_request(url, params)

In [125]:
def get_request(url, params):
    res = requests.get(url, params)
    sc = res.status_code
    return (sc, res)

In [126]:
def make_before_vals_df(value_ama_s, value_ama_c, value_ar_s, value_ar_c, verbose=True):
    before_list = [['AMA', 'Submissions', value_ama_s],
                   ['AMA', 'Comments', value_ama_c],
                   ['AskReddit', 'Submissions', value_ar_s],
                   ['AskReddit', 'Comments', value_ar_c]]
    before = pd.DataFrame(before_list, columns=['subreddit', 'sub_com', 'before_value'])
    before['before_value'] = '_' + before['before_value'].apply(str)
    
    if(verbose):
        print('\n')
        print(datetime.datetime.now().strftime('%Y-%m-%d, %H%M - finish:'))
        print(f'BEFORE VAL, ama submissions: {value_ama_s}')
        print(f'BEFORE VAL, ama comments:    {value_ama_c}')
        print(f'BEFORE VAL, ar submissions:  {value_ar_s}')
        print(f'BEFORE VAL, ar comments:     {value_ar_c}')
    return before

In [127]:
def write_to_csv(df_output, sr, sub_com):
    if not os.path.exists('../data/'):
        os.makedirs('../data/')
    fn = datetime.datetime.now().strftime('%Y-%m-%d_%H%M_')
    output_file = '../data/' + fn + sr + '_' + sub_com + '.csv'
    df_output.to_csv(output_file, index=False)

In [128]:
call_scraper(100)

READ IN BEFORE VALS: 
0    1619194268
1    1619220423
2    1619238568
3    1619222669
Name: before_value, dtype: int64
AMA STATUS (submission): 200
AMA STATUS (comment): 200
AskReddit STATUS (submission): 200
AskReddit STATUS (comment): 200


2021-04-24, 0113 - finish:
BEFORE VAL, ama submissions: 1619143003
BEFORE VAL, ama comments:    1619217435
BEFORE VAL, ar submissions:  1619237456
BEFORE VAL, ar comments:     1619222633


In [129]:
call_scraper(100)

READ IN BEFORE VALS: 
0    1619143003
1    1619217435
2    1619237456
3    1619222633
Name: before_value, dtype: int64
AMA STATUS (submission): 200
AMA STATUS (comment): 200
AskReddit STATUS (submission): 200
AskReddit STATUS (comment): 200


2021-04-24, 0114 - finish:
BEFORE VAL, ama submissions: 1619098454
BEFORE VAL, ama comments:    1619214064
BEFORE VAL, ar submissions:  1619236392
BEFORE VAL, ar comments:     1619222596


In [130]:
for i in range(22):
    call_scraper(100)

READ IN BEFORE VALS: 
0    1619098454
1    1619214064
2    1619236392
3    1619222596
Name: before_value, dtype: int64
AMA STATUS (submission): 200
AMA STATUS (comment): 200
AskReddit STATUS (submission): 200
AskReddit STATUS (comment): 200


2021-04-24, 0118 - finish:
BEFORE VAL, ama submissions: 1619048789
BEFORE VAL, ama comments:    1619211515
BEFORE VAL, ar submissions:  1619235363
BEFORE VAL, ar comments:     1619222563
READ IN BEFORE VALS: 
0    1619048789
1    1619211515
2    1619235363
3    1619222563
Name: before_value, dtype: int64
AMA STATUS (submission): 200
AMA STATUS (comment): 200
AskReddit STATUS (submission): 200
AskReddit STATUS (comment): 200


2021-04-24, 0118 - finish:
BEFORE VAL, ama submissions: 1619014836
BEFORE VAL, ama comments:    1619209544
BEFORE VAL, ar submissions:  1619234516
BEFORE VAL, ar comments:     1619222524
READ IN BEFORE VALS: 
0    1619014836
1    1619209544
2    1619234516
3    1619222524
Name: before_value, dtype: int64
AMA STATUS (submissio